# Colorado Scraping 

In [1]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [2]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [18]:
# state - UPDATE EACH TIME
state = 'colorado'

In [4]:
# review of robots.txt - nonexistent

## Scrape Links
`Beautiful Soup`

### Test on one link

In [4]:
# access webpage
link = 'https://www.colorado.gov/governor/news?page=1'
server_response = requests.get(link, header_list)
server_response

<Response [200]>

In [11]:
# parse and isolate content
soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
#soup
results = soup.find_all('article') 
# results
# title = [i.find('h2', class_ = 'press-release__title').text for i in results]
# title_clean = [i.replace(u'\n', u'') for i in title]
# title_clean
# date = [i.find('footer', class_ = 'press-release__date').text for i in results]
# date

['Thursday, February 1, 2024',
 'Thursday, February 1, 2024',
 'Wednesday, January 31, 2024',
 'Wednesday, January 31, 2024',
 'Wednesday, January 31, 2024',
 'Wednesday, January 31, 2024',
 'Monday, January 29, 2024',
 'Monday, January 29, 2024',
 'Friday, January 26, 2024',
 'Friday, January 26, 2024',
 'Wednesday, January 24, 2024']

### Scrape all links

In [14]:
# define webpage scraping 
def scrape_colorado_links(website, headers):
    server_response = requests.get(website, headers)
    soup = bs4.BeautifulSoup(server_response.content, features="html.parser")
    results = soup.find_all('article') 
    results_list = [{'title': i.find('h2', class_ = 'press-release__title').text,
                 'date': i.find('footer', class_ = 'press-release__date').text, 
                'link': i.find('a')['href']} for i in results]
    df = pd.DataFrame.from_dict(results_list)
    df.title = [i.replace(u'\n', u'') for i in df.title]
    df['content'] = np.nan
    return df

In [13]:
scrape_colorado_links('https://www.colorado.gov/governor/news?page=1', header_list)

,title,date,link,content
0,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",/governor/news/11471-governor-polis-and-colora...,NaN
1,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",/governor/news/11471-governor-polis-and-colora...,NaN
2,Governor Polis Appoints James H. Hesson to the...,"Wednesday, January 31, 2024",/governor/news/11466-governor-polis-appoints-j...,NaN
3,Governor Polis Signs First Law of the 2024 Leg...,"Wednesday, January 31, 2024",/governor/news/11456-governor-polis-signs-firs...,NaN
4,Colorado and Alberta Announce Energy and Clima...,"Wednesday, January 31, 2024",/governor/news/11461-colorado-and-alberta-anno...,NaN
5,Laying Groundwork for More Housing That is Aff...,"Wednesday, January 31, 2024",/governor/news/11451-laying-groundwork-more-ho...,NaN
6,Governor Polis Delivers Regional State of the ...,"Monday, January 29, 2024",/governor/news/11446-governor-polis-delivers-r...,NaN
7,Colorado and Wyoming Awarded Inaugural NSF Eng...,"Monday, January 29, 2024",/governor/news/11441-colorado-and-wyoming-awar...,NaN
8,Polis Administration Announces New Curated Exh...,"Friday, January 26, 2024",/governor/news/11431-polis-administration-anno...,NaN
9,Governor Polis Delivers Regional State of the ...,"Friday, January 26, 2024",/governor/news/11426-governor-polis-delivers-r...,NaN


In [16]:
# try to scrape the first set of pages
# manual review for 2023 - I want pages 1-42, give or take a few links on first and last pages
df_list=[]

for i in range(1,43):
    try: 
        df = scrape_colorado_links(f'https://www.colorado.gov/governor/news?page={i}', 
                                  header_list)
        df['page_scraped'] = i 
        df_list.append(df)
    except Exception as e:
        print(f'Error: {e} at page {i}')

In [19]:
# combine into one df
combined = pd.concat(df_list, ignore_index=True)
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         462 non-null    object 
 1   date          462 non-null    object 
 2   link          462 non-null    object 
 3   content       0 non-null      float64
 4   page_scraped  462 non-null    int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 18.2+ KB


In [24]:
combined.head()

,title,date,link,content,page_scraped
0,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",/governor/news/11471-governor-polis-and-colora...,NaN,1
1,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",/governor/news/11471-governor-polis-and-colora...,NaN,1
2,Governor Polis Appoints James H. Hesson to the...,"Wednesday, January 31, 2024",/governor/news/11466-governor-polis-appoints-j...,NaN,1
3,Governor Polis Signs First Law of the 2024 Leg...,"Wednesday, January 31, 2024",/governor/news/11456-governor-polis-signs-firs...,NaN,1
4,Colorado and Alberta Announce Energy and Clima...,"Wednesday, January 31, 2024",/governor/news/11461-colorado-and-alberta-anno...,NaN,1


In [25]:
# seems to be missing the url prefix so let's add this to the links 
combined.link = ['https://www.colorado.gov' + i for i in combined.link]
combined.head()

In [27]:
combined.to_csv(f'{state}_links.csv', index = False)

In [28]:
combined = pd.read_csv(f'{state}_links.csv')
combined.head()

,title,date,link,content,page_scraped
0,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",https://www.colorado.gov/governor/news/11471-g...,NaN,1
1,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",https://www.colorado.gov/governor/news/11471-g...,NaN,1
2,Governor Polis Appoints James H. Hesson to the...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11466-g...,NaN,1
3,Governor Polis Signs First Law of the 2024 Leg...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11456-g...,NaN,1
4,Colorado and Alberta Announce Energy and Clima...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11461-c...,NaN,1


## Scrape content from links
`Beautiful Soup`

### Test with one link

In [37]:
# define link scraping 
def get_colorado_content(link, df, header_list):
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                df['content']
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('div', 
                                        class_ = 'field field--name-body field--type-text-with-summary field--label-hidden field__item').find_all('p')
                string_result = [str(i.text) for i in result]
                string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
                string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
                result_merged = " ".join(string_result_clean)
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [30]:
# access webpage
link = 'https://www.colorado.gov/governor/news/9341-state-government-offices-denver-area-will-open-late-today'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [36]:
# parse and isolate content
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('div', class_ = 'field field--name-body field--type-text-with-summary field--label-hidden field__item').find_all('p')
string_result = [str(i.text) for i in result]
string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
result_merged = " ".join(string_result_clean)
result_merged

' DENVER - State of Colorado government offices in the Denver and surrounding suburban counties will open to the public at 10:00 a.m. Thursday, December 29th, due to extreme weather conditions. There will be a 2-hour delay in start times for state employees to arrive at work safely.  State facilities essential to public health and safety will maintain regular operating schedules and “essential personnel” must report to work at their normal scheduled time.  The Colorado Judicial Branch posts its own information regarding closures and delays related to Colorado’s courthouses and probation offices. Visit www.courts.state.co.us for current information related to courts and probation. The Colorado General Assembly posts daily schedules at www.leg.colorado.gov.  ### '

### Scrape content from all links

In [39]:
# test on 5 to see if it works
for i in combined.link[0:6]:
     get_colorado_content(i, combined, header_list)
    
combined.head()

'content already existing for https://www.colorado.gov/governor/news/11471-governor-polis-and-colorado-energy-office-celebrate-new-tax-incentives-lower-cost-high'

'content already existing for https://www.colorado.gov/governor/news/11471-governor-polis-and-colorado-energy-office-celebrate-new-tax-incentives-lower-cost-high'

'content already existing for https://www.colorado.gov/governor/news/11466-governor-polis-appoints-james-h-hesson-moffat-county-court-14th-judicial-district'

'content already existing for https://www.colorado.gov/governor/news/11456-governor-polis-signs-first-law-2024-legislative-session-help-save-coloradans-money'

'content already existing for https://www.colorado.gov/governor/news/11461-colorado-and-alberta-announce-energy-and-climate-partnership'

'content already existing for https://www.colorado.gov/governor/news/11451-laying-groundwork-more-housing-affordable-polis-administration-announces-first'

,title,date,link,content,page_scraped
0,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",https://www.colorado.gov/governor/news/11471-g...,"DENVER -Today, Governor Polis and the Colorado...",1
1,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",https://www.colorado.gov/governor/news/11471-g...,"DENVER -Today, Governor Polis and the Colorado...",1
2,Governor Polis Appoints James H. Hesson to the...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11466-g...,"DENVER -Today, Governor Polis appointed James ...",1
3,Governor Polis Signs First Law of the 2024 Leg...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11456-g...,"DENVER -Today, Governor Polis signedHB24-1084 ...",1
4,Colorado and Alberta Announce Energy and Clima...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11461-c...,"DENVER- Today, Colorado Governor Jared Polis a...",1


In [40]:
# do for all
for i in combined.link:
     get_colorado_content(i, combined, header_list)

'content already existing for https://www.colorado.gov/governor/news/11471-governor-polis-and-colorado-energy-office-celebrate-new-tax-incentives-lower-cost-high'

'content already existing for https://www.colorado.gov/governor/news/11471-governor-polis-and-colorado-energy-office-celebrate-new-tax-incentives-lower-cost-high'

'content already existing for https://www.colorado.gov/governor/news/11466-governor-polis-appoints-james-h-hesson-moffat-county-court-14th-judicial-district'

'content already existing for https://www.colorado.gov/governor/news/11456-governor-polis-signs-first-law-2024-legislative-session-help-save-coloradans-money'

'content already existing for https://www.colorado.gov/governor/news/11461-colorado-and-alberta-announce-energy-and-climate-partnership'

...

'success: content added from https://www.colorado.gov/governor/news/9316-gov-polis-visits-warming-centers-shelters-supporting-coloradans-migrants-state-continues'

'success: content added from https://www.colorado.gov/governor/news/9311-governor-polis-announces-clemency-24-individuals'

'success: content added from https://www.colorado.gov/governor/news/9306-gov-polis-announces-boards-and-commissions-appointments'

'success: content added from https://www.colorado.gov/governor/news/9301-gov-polis-activates-colorado-national-guard-help-assist-extreme-cold-weather'

'success: content added from https://www.colorado.gov/governor/news/9296-provider-registration-open-universal-preschool-upk-colorado'

## Validate and Clean

In [42]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,date,link,content,page_scraped


In [48]:
# convert date to date-time object to subset
combined['date_clean'] = pd.to_datetime(combined['date'])
combined.head()

,title,date,link,content,page_scraped,date_clean
0,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",https://www.colorado.gov/governor/news/11471-g...,"DENVER -Today, Governor Polis and the Colorado...",1,2024-02-01
1,Governor Polis and the Colorado Energy Office ...,"Thursday, February 1, 2024",https://www.colorado.gov/governor/news/11471-g...,"DENVER -Today, Governor Polis and the Colorado...",1,2024-02-01
2,Governor Polis Appoints James H. Hesson to the...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11466-g...,"DENVER -Today, Governor Polis appointed James ...",1,2024-01-31
3,Governor Polis Signs First Law of the 2024 Leg...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11456-g...,"DENVER -Today, Governor Polis signedHB24-1084 ...",1,2024-01-31
4,Colorado and Alberta Announce Energy and Clima...,"Wednesday, January 31, 2024",https://www.colorado.gov/governor/news/11461-c...,"DENVER- Today, Colorado Governor Jared Polis a...",1,2024-01-31


In [49]:
# drop original date column 
combined_cl = combined.drop(columns=['date']).copy()

In [50]:
# subset to 2023 only 
combined_2023 = combined_cl[(combined_cl['date_clean'] >= '2023-01-01')
                     & (combined_cl['date_clean'] <= '2023-12-31')].copy()

In [51]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 407 documents from colorado for 2023


## Export 

In [52]:
# export
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [53]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,title,link,content,page_scraped,date_clean
0,Delivering Real Results for Coloradans: New La...,https://www.colorado.gov/governor/news/11286-d...,"New tax credits for up to $7,500 on electric ...",5,2023-12-29
1,Delivering Real Results for Coloradans: New La...,https://www.colorado.gov/governor/news/11286-d...,"New tax credits for up to $7,500 on electric ...",5,2023-12-29
2,Governor Polis Announces Technical Assistance ...,https://www.colorado.gov/governor/news/11261-g...,"DENVER— Today Governor Polis, in partnership w...",5,2023-12-28
3,Governor Polis Announces Final Revitalizing Ma...,https://www.colorado.gov/governor/news/11256-g...,"STATEWIDE - Today, Governor Polis and the Colo...",5,2023-12-27
4,Governor Polis Goes “Platinum”,https://www.colorado.gov/governor/news/11251-g...,Governor Polis Celebrates a Holiday Classic w...,5,2023-12-26
